# PyPSA Components Basics

This tutorial covers the basic components available in PyPSA and how to configure them. We'll explore generators, loads, lines, and other essential components.

## Basic Components Overview

PyPSA supports several types of components:

1. **Buses**: Network nodes
2. **Generators**: Power plants and other generation units
3. **Loads**: Electricity demand
4. **Lines**: AC transmission lines
5. **Links**: DC transmission lines or other connections
6. **Stores**: Storage units
7. **Carriers**: Energy carriers (e.g., electricity, gas, heat)

Let's create a network and explore each component type:

In [ ]:
import pypsa
import pandas as pd
import numpy as np

# Create network with time series
network = pypsa.Network()
network.set_snapshots(pd.date_range("2024-01-01", periods=24, freq="h"))

# Add buses
for i in range(3):
    network.add("Bus", f"bus {i}", v_nom=20.0)

# Add carriers
network.add("Carrier", "gas", co2_emissions=0.2)
network.add("Carrier", "solar")
network.add("Carrier", "wind")

## Generators

Generators represent power plants or other generation units. They can be configured with various parameters:

In [ ]:
# Add a gas generator
network.add(
    "Generator",
    "gas_plant",
    bus="bus 0",
    p_nom=100,  # Nominal power capacity
    p_nom_extendable=True,  # Allow capacity expansion
    marginal_cost=50,  # Operating cost per MWh
    capital_cost=1000,  # Investment cost per MW
    carrier="gas",
    efficiency=0.4  # Conversion efficiency
)

# Add a solar generator with variable output
solar_profile = pd.Series(
    [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
     0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    index=network.snapshots
)

network.add(
    "Generator",
    "solar",
    bus="bus 1",
    p_nom=50,
    p_nom_extendable=True,
    p_max_pu=solar_profile,  # Maximum power output as fraction of p_nom
    marginal_cost=0,  # Zero marginal cost for renewables
    capital_cost=800,
    carrier="solar"
)

print("Generators in the network:")
print(network.generators)

## Loads

Loads represent electricity demand. They can have fixed or time-varying demand profiles:

In [ ]:
# Create a load profile
load_profile = pd.Series(
    [0.6, 0.5, 0.4, 0.4, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.7,
     0.7, 0.7, 0.7, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.6, 0.5, 0.5],
    index=network.snapshots
)

# Add loads with different profiles
network.add(
    "Load",
    "load_1",
    bus="bus 1",
    p_set=50 * load_profile  # Time-varying demand
)

network.add(
    "Load",
    "load_2",
    bus="bus 2",
    p_set=30  # Fixed demand
)

print("Loads in the network:")
print(network.loads)

## Lines and Links

Lines represent AC transmission connections, while Links can represent DC lines or other connections:

In [ ]:
# Add AC transmission lines
for i in range(3):
    network.add(
        "Line",
        f"line_{i}",
        bus0=f"bus {i}",
        bus1=f"bus {(i + 1) % 3}",
        x=0.1,  # Reactance
        r=0.01,  # Resistance
        s_nom=100,  # Nominal apparent power
        s_nom_extendable=True,  # Allow capacity expansion
        capital_cost=1000  # Investment cost per MVA
    )

# Add a DC link
network.add(
    "Link",
    "dc_link",
    bus0="bus 0",
    bus1="bus 2",
    p_nom=50,
    p_nom_extendable=True,
    capital_cost=2000,
    efficiency=0.95  # DC link efficiency
)

print("Lines in the network:")
print(network.lines)

print("\nLinks in the network:")
print(network.links)

## Stores

Stores represent storage units that can charge and discharge:

In [ ]:
# Add a storage unit
network.add(
    "Store",
    "battery",
    bus="bus 1",
    e_nom=100,  # Nominal energy capacity
    e_nom_extendable=True,  # Allow capacity expansion
    e_cyclic=True,  # Allow cyclic operation
    e_min_pu=0.1,  # Minimum energy level as fraction of e_nom
    p_nom=20,  # Nominal power capacity
    p_nom_extendable=True,  # Allow power capacity expansion
    capital_cost_e=100,  # Investment cost per MWh
    capital_cost_p=1000,  # Investment cost per MW
    marginal_cost=0.1  # Operating cost per MWh
)

print("Stores in the network:")
print(network.stores)

## Component Attributes

Key attributes for different components:

### Generators
- `p_nom`: Nominal power capacity
- `p_nom_extendable`: Whether capacity can be expanded
- `marginal_cost`: Operating cost per MWh
- `capital_cost`: Investment cost per MW
- `efficiency`: Conversion efficiency
- `p_max_pu`: Maximum power output as fraction of p_nom

### Loads
- `p_set`: Power demand (fixed or time series)

### Lines
- `s_nom`: Nominal apparent power
- `x`: Reactance
- `r`: Resistance
- `capital_cost`: Investment cost per MVA

### Stores
- `e_nom`: Nominal energy capacity
- `p_nom`: Nominal power capacity
- `e_cyclic`: Whether storage can be cycled
- `e_min_pu`: Minimum energy level
- `capital_cost_e`: Investment cost per MWh
- `capital_cost_p`: Investment cost per MW

## Next Steps

In the next tutorial, we'll learn how to visualize and analyze these components in the network.